<a href="https://colab.research.google.com/github/Mr-Q8/Curso.Prep.Henry/blob/master/_https_www_youtube_com_ElArchivoProhibido_y4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LTX-VIDEO Text to Video**

- You can use the free T4 GPU to run this depending on the output video resolution and number of frames. The default setting runs without issues, but at 768 by 512 output resolution with 121 frames, the decoding process crashes the 12.7GB RAM.  For faster video generation with higher resolutions and frames, use higher GPUs.
- If you want to generate a video with n frames, then set frames to n+1. e.g. To generate a video with 72 frames, set frames to 73.
- You need to use detailed prompts to get decent results.
- Videos are generated at 24fps.

In [1]:
# @title Prepare Environment
!pip install torch==2.6.0 torchvision==0.21.0
%cd /content
Always_Load_Models_for_Inference = False
Use_t5xxl_fp16 = False
# Install dependencies
!pip install -q torchsde einops diffusers accelerate xformers==0.0.29.post2
!pip install av
!git clone https://github.com/Isi-dev/ComfyUI
%cd /content/ComfyUI
!apt -y install -qq aria2 ffmpeg

# Download required models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/ltx-video-2b-v0.9.5.safetensors -d /content/ComfyUI/models/checkpoints -o ltx-video-2b-v0.9.5.safetensors
if Use_t5xxl_fp16:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp16.safetensors
else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp8_e4m3fn_scaled.safetensors

# Initial setup
import torch
import numpy as np
from PIL import Image
import gc
import sys
import random
import os
import imageio
from google.colab import files
from IPython.display import display, HTML
sys.path.insert(0, '/content/ComfyUI')

from comfy import model_management

from nodes import (
    CheckpointLoaderSimple,
    CLIPLoader,
    CLIPTextEncode,
    VAEDecode
)

from comfy_extras.nodes_custom_sampler import (
    KSamplerSelect,
    SamplerCustom
)

from comfy_extras.nodes_lt import (
    LTXVConditioning,
    LTXVScheduler,
    EmptyLTXVLatentVideo
)

checkpoint_loader = CheckpointLoaderSimple()
clip_loader = CLIPLoader()
clip_encode_positive = CLIPTextEncode()
clip_encode_negative = CLIPTextEncode()
scheduler = LTXVScheduler()
sampler_select = KSamplerSelect()
conditioning = LTXVConditioning()
empty_latent_video = EmptyLTXVLatentVideo()
sampler = SamplerCustom()
vae_decode = VAEDecode()

# if not Always_Load_Models_for_Inference:
# with torch.inference_mode():
#     # Load models
#     print("Loading Model...")
#     model, _, vae = checkpoint_loader.load_checkpoint("ltx-video-2b-v0.9.5.safetensors")
#     print("Loaded model!")
#     # print("Loading Text_Encoder...")
#     # clip = clip_loader.load_clip("t5xxl_fp8_e4m3fn_scaled.safetensors", "ltxv", "default")[0]
#     # print("Loaded Text_Encoder!")


def clear_memory():
    """Frees GPU (VRAM) and CPU RAM memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    for obj in list(globals().values()):
        if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
            del obj

    gc.collect()

def generate_video(
    positive_prompt: str = "A drone quickly rises through a bank of morning fog...",
    negative_prompt: str = "low quality, worst quality...",
    width: int = 768,
    height: int = 512,
    seed: int = 0,
    steps: int = 30,
    cfg_scale: float = 2.05,
    sampler_name: str = "res_multistep",
    length: int = 49,
    fps: int = 24
):

    with torch.inference_mode():
        print("Loading Text_Encoder...")
        clip = clip_loader.load_clip("t5xxl_fp8_e4m3fn_scaled.safetensors", "ltxv", "default")[0]
        print("Loaded Text_Encoder!")

    try:
        assert width % 32 == 0, "Width must be divisible by 32"
        assert height % 32 == 0, "Height must be divisible by 32"

        positive = clip_encode_positive.encode(clip, positive_prompt)[0]
        negative = clip_encode_negative.encode(clip, negative_prompt)[0]

        del clip
        torch.cuda.empty_cache()
        gc.collect()
        print("Text_Encoder removed from memory")

        empty_latent = empty_latent_video.generate(width, height, length)[0]

        sigmas = scheduler.get_sigmas(steps, cfg_scale, 0.95, True, 0.1)[0]
        selected_sampler = sampler_select.get_sampler(sampler_name)[0]
        conditioned = conditioning.append(positive, negative, 25.0)

        print("Loading model & VAE...")
        model, _, vae = checkpoint_loader.load_checkpoint("ltx-video-2b-v0.9.5.safetensors")
        print("Loaded model & VAE!")

        print("Generating video...")
        sampled = sampler.sample(
            model=model,
            add_noise=True,
            noise_seed=seed if seed != 0 else random.randint(0, 2**32),
            cfg=cfg_scale,
            positive=conditioned[0],
            negative=conditioned[1],
            sampler=selected_sampler,
            sigmas=sigmas,
            latent_image=empty_latent
        )[0]

        del model
        torch.cuda.empty_cache()
        gc.collect()
        print("Model removed from memory")

        with torch.no_grad():
          try:
              print("Decodimg Latents...")
              decoded = vae_decode.decode(vae, sampled)[0].detach()
              print("Latents Decoded!")
              del vae
              torch.cuda.empty_cache()
              gc.collect()
              print("VAE removed from memory")

              output_path = "/content/output.mp4"
              with imageio.get_writer(output_path, fps=fps) as writer:
                  for i, frame in enumerate(decoded):
                      frame_np = (frame.cpu().numpy() * 255).astype(np.uint8)
                      writer.append_data(frame_np)
                      if i % 10 == 0:  # Periodic cleanup
                          torch.cuda.empty_cache()

              print(f"Successfully processed {len(decoded)} frames")


          except Exception as e:
              print(f"Decoding error: {str(e)}")
              raise

        print("Displaying Video...")
        display_video(output_path)

    except Exception as e:
        print(f"Video generation failed: {str(e)}")
        raise
    finally:
        clear_memory()

def display_video(video_path):
    from IPython.display import HTML
    from base64 import b64encode

    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    display(HTML(f"""
    <video width=512 controls autoplay loop>
        <source src="{data_url}" type="video/mp4">
    </video>
    """))

print("✅ Environment Setup Complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
# @title Prepare Environment and Install Server Libraries
!pip install torch==2.6.0 torchvision==0.21.0
# %cd /content # Comentamos esto si queremos que el server corra en la raíz
Always_Load_Models_for_Inference = False
Use_t5xxl_fp16 = False
    # Install dependencies for video generation
!pip install -q torchsde einops diffusers accelerate xformers==0.0.29.post2
!pip install av
!git clone https://github.com/Isi-dev/ComfyUI /content/ComfyUI # Clonar ComfyUI en una ubicación conocida
    # %cd /content/ComfyUI # Comentamos esto para mantener la raíz como directorio de trabajo del server
!apt -y install -qq aria2 ffmpeg

    # Download required models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/ltx-video-2b-v0.9.5.safetensors -d /content/ComfyUI/models/checkpoints -o ltx-video-2b-v0.9.5.safetensors
if Use_t5xxl_fp16:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp16.safetensors
else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp8_e4m3fn_scaled.safetensors

    # --- Install Server Libraries ---
!pip install fastapi uvicorn pyngrok google-cloud-storage

    # Initial setup for video generation code
import torch
import numpy as np
from PIL import Image
import gc
import sys
import random
import os
import imageio
    # from google.colab import files # No necesario en el servidor
    # from IPython.display import display, HTML # No necesario en el servidor

    # Add ComfyUI directory to sys.path
sys.path.insert(0, '/content/ComfyUI')

from comfy import model_management

from nodes import (
        CheckpointLoaderSimple,
        CLIPLoader,
        CLIPTextEncode,
        VAEDecode
    )

from comfy_extras.nodes_custom_sampler import (
        KSamplerSelect,
        SamplerCustom
    )

from comfy_extras.nodes_lt import (
        LTXVConditioning,
        LTXVScheduler,
        EmptyLTXVLatentVideo
    )

    # Initialize nodes
checkpoint_loader = CheckpointLoaderSimple()
clip_loader = CLIPLoader()
clip_encode_positive = CLIPTextEncode()
clip_encode_negative = CLIPTextEncode()
scheduler = LTXVScheduler()
sampler_select = KSamplerSelect()
conditioning = LTXVConditioning()
empty_latent_video = EmptyLTXVLatentVideo()
sampler = SamplerCustom()
vae_decode = VAEDecode()

    # --- Google Cloud Storage Upload Function ---
from google.cloud import storage

def upload_video_to_gcs(source_file_name, destination_blob_name, bucket_name):
        """Sube un archivo a un bucket de Cloud Storage."""
        # Colab generalmente usa las credenciales de usuario, que deberían tener acceso a GCS si el usuario lo tiene.
        # Si esto falla, puede que necesites autenticarte explícitamente usando una cuenta de servicio.
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_filename(source_file_name)

        print(f"Archivo {source_file_name} subido a {destination_blob_name} en el bucket {bucket_name}.")
        return f"gs://{bucket_name}/{destination_blob_name}" # Retornar la ruta GS

    # --- Modified Video Generation Function to Save to GCS ---
def generate_video(
        positive_prompt: str,
        negative_prompt: str,
        width: int,
        height: int,
        seed: int,
        steps: int,
        cfg_scale: float,
        sampler_name: str,
        length: int,
        fps: int,
        output_bucket_name: str, # Parameter for the output GCS bucket name
        output_blob_prefix: str # Parameter for the prefix of the output file name in GCS
    ):

        with torch.inference_mode():
            print("Loading Text_Encoder...")
            # Assuming models are downloaded to /content/ComfyUI/models
            clip = clip_loader.load_clip("/content/ComfyUI/models/text_encoders/t5xxl_fp8_e4m3fn_scaled.safetensors", "ltxv", "default")[0]
            print("Loaded Text_Encoder!")

        try:
            assert width % 32 == 0, "Width must be divisible by 32"
            assert height % 32 == 0, "Height must be divisible by 32"

            positive = clip_encode_positive.encode(clip, positive_prompt)[0]
            negative = clip_encode_negative.encode(clip, negative_prompt)[0]

            del clip
            torch.cuda.empty_cache()
            gc.collect()
            print("Text_Encoder removed from memory")

            empty_latent = empty_latent_video.generate(width, height, length)[0]

            sigmas = scheduler.get_sigmas(steps, cfg_scale, 0.95, True, 0.1)[0]
            selected_sampler = sampler_select.get_sampler(sampler_name)[0]
            conditioned = conditioning.append(positive, negative, 25.0)

            print("Loading model & VAE...")
            # Assuming models are downloaded to /content/ComfyUI/models
            model, _, vae = checkpoint_loader.load_checkpoint("/content/ComfyUI/models/checkpoints/ltx-video-2b-v0.9.5.safetensors")
            print("Loaded model & VAE!")

            print("Generando video...")
            sampled = sampler.sample(
                model=model,
                add_noise=True,
                noise_seed=seed if seed != 0 else random.randint(0, 2**32),
                cfg=cfg_scale,
                positive=conditioned[0],
                negative=conditioned[1],
                sampler=selected_sampler,
                sigmas=sigmas,
                latent_image=empty_latent
            )[0]

            del model
            torch.cuda.empty_cache()
            gc.collect()
            print("Model removed from memory")

            with torch.no_grad():
              try:
                  print("Decodificando Latents...")
                  decoded = vae_decode.decode(vae, sampled)[0].detach()
                  print("Latents Decoded!")
                  del vae
                  torch.cuda.empty_cache()
                  gc.collect()
                  print("VAE removed from memory")

                  # --- Save video to temporary file and upload to GCS ---
                  temp_output_path = "/tmp/output.mp4" # Save temporarily
                  with imageio.get_writer(temp_output_path, fps=fps) as writer:
                      for i, frame in enumerate(decoded):
                          frame_np = (frame.cpu().numpy() * 255).astype(np.uint8)
                          writer.append_data(frame_np)
                          if i % 10 == 0:  # Periodic cleanup
                              torch.cuda.empty_cache()

                  print(f"Successfully processed {len(decoded)} frames")

                  # Generate a unique name for the file in GCS
                  import uuid
                  gcs_blob_name = f"{output_blob_prefix}/{uuid.uuid4().hex}.mp4"

                  # Upload the video to Cloud Storage
                  video_gcs_path = upload_video_to_gcs(temp_output_path, gcs_blob_name, output_bucket_name)

                  print(f"Video generated and uploaded to Cloud Storage: {video_gcs_path}")

                  # Clean up the temporary local file
                  os.remove(temp_output_path)
                  print(f"Removed temporary file: {temp_output_path}")

                  return video_gcs_path # Return the GCS path

              except Exception as e:
                  print(f"Decoding error: {str(e)}")
                  raise

        except Exception as e:
            print(f"Video generation failed: {str(e)}")
            raise
        finally:
            clear_memory() # Final memory cleanup


fatal: destination path '/content/ComfyUI' already exists and is not an empty directory.
aria2 is already the newest version (1.36.0-1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b52173|OK  |       0B/s|/content/ComfyUI/models/checkpoints/ltx-video-2b-v0.9.5.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
35b864|OK  |       0B/s|/content/ComfyUI/models/text_encoders/t5xxl_fp8_e4m3fn_scaled.safetensors

Status Legend:
(OK):download completed.


In [5]:
# --- FastAPI Server Setup ---
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import os

    # Pydantic model for request body validation
class VideoRequest(BaseModel):
        positive_prompt: str
        negative_prompt: str
        width: int
        height: int
        seed: int = 0
        steps: int = 25
        cfg_scale: float = 2.05
        sampler_name: str = "res_multistep"
        length: int = 73
        fps: int = 24
        output_bucket_name: str # Name of the GCS bucket
        output_blob_prefix: str = "generated_videos" # Prefix for the file name in GCS

app = FastAPI()

@app.post("/generate-video")
async def generate_video_endpoint(request: VideoRequest):
        """Receives video generation parameters and returns the GCS path of the generated video."""
        try:
            print("Received video generation request:", request)

            video_gcs_path = generate_video(
                positive_prompt=request.positive_prompt,
                negative_prompt=request.negative_prompt,
                width=request.width,
                height=request.height,
                seed=request.seed,
                steps=request.steps,
                cfg_scale=request.cfg_scale,
                sampler_name=request.sampler_name,
                length=request.length,
                fps=request.fps,
                output_bucket_name=request.output_bucket_name,
                output_blob_prefix=request.output_blob_prefix
            )

            return {"video_gcs_path": video_gcs_path}

        except Exception as e:
            print(f"Error in generate_video_endpoint: {str(e)}")
            raise HTTPException(status_code=500, detail=f"Video generation failed: {str(e)}")


In [8]:
# --- Pyngrok and Uvicorn Setup to run the FastAPI app ---\
    # Make Colab compatible with asyncio
nest_asyncio.apply()

    # Configure ngrok auth token (Replace with your actual ngrok token or read from a secret)
    # You can get an ngrok token from your ngrok dashboard after signing up.
    # conf.get_default().auth_token =  "YOUR_NGROK_AUTH_TOKEN"\
    # WARNING: Storing tokens directly in the notebook is NOT secure. Use Colab Secrets if possible.
try:
        from google.colab import userdata
        NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN') # Assuming you saved your ngrok token as 'NGROK_AUTH_TOKEN' in Colab Secrets
        ngrok.set_auth_token(NGROK_AUTH_TOKEN) # Use set_auth_token instead of conf

except:
        print("Colab Secrets not available or NGROK_AUTH_TOKEN not found. Pyngrok might not work without auth token.")
        print("Please add your ngrok auth token as a Colab Secret named 'NGROK_AUTH_TOKEN'.")

    # Define the port for the FastAPI app
COLAB_PORT = 8000 # Or any other available port

    # Start ngrok tunnel
print(f"Starting ngrok tunnel for port {COLAB_PORT}...")
    # Disconnect existing tunnels to avoid conflicts
ngrok.kill()
public_url = ngrok.connect(COLAB_PORT).public_url
print(f"🔥 Public ngrok URL: {public_url}")

    # --- IMPORTANT --- #
    # You need to send this public_url to your backend (e.g., Firebase Realtime Database or Firestore)\
    # so your Cloud Function knows where to send requests.\
    # Example (requires Firebase Admin SDK setup in Colab, which adds complexity):\
    # from firebase_admin import db # Example for Realtime Database\
    # db.reference('/colab_server_url').set(public_url)\
    # For simplicity now, you'll manually copy this URL to your backend config.\
    # ----------------- #

    # Run the FastAPI app using uvicorn
print(f"Running FastAPI app on port {COLAB_PORT}...")
uvicorn.run(app, host="0.0.0.0", port=COLAB_PORT)

    # This cell will block execution as it runs the server. You'll need to stop it manually if running interactively.
    # For automation, you might need to adjust how uvicorn is run or explore other ways to keep the Colab session alive and the server running.


Starting ngrok tunnel for port 8000...
🔥 Public ngrok URL: https://089a4f5c510f.ngrok-free.app
Running FastAPI app on port 8000...


INFO:     Started server process [622]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [622]


In [ ]:
# ==============================================================================
#  PASO 1: Instalar todas las librerías necesarias
# ==============================================================================
!pip install fastapi "uvicorn[standard]" pyngrok nest_asyncio cloudinary -q

# ==============================================================================
#  PASO 2: Importar librerías y configurar el servidor
# ==============================================================================
import os
import asyncio
from fastapi import FastAPI, HTTPException
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Importaciones de Cloudinary y para leer los Secrets de Colab
import cloudinary
import cloudinary.uploader
from google.colab import userdata

# --- Configuración de Cloudinary ---
# El código lee las claves de forma segura desde los "Secrets" de Colab
try:
    cloudinary.config(
      cloud_name = userdata.get('CLOUDINARY_CLOUD_NAME'),
      api_key = userdata.get('CLOUDINARY_API_KEY'),
      api_secret = userdata.get('CLOUDINARY_API_SECRET'),
      secure = True
    )
    print("✅ Configuración de Cloudinary exitosa.")
except Exception as e:
    print(f"❌ Error al configurar Cloudinary. Asegúrate de que los secrets (CLOUDINARY_CLOUD_NAME, CLOUDINARY_API_KEY, CLOUDINARY_API_SECRET) están bien guardados. Error: {e}")

# --- Definición de la aplicación FastAPI ---
app = FastAPI()

@app.post("/generate-video/")
async def generate_video_endpoint(prompt: str):
    """
    Endpoint que simula la creación de un video, lo sube a Cloudinary
    y devuelve la URL pública segura.
    """
    try:
        print(f"▶️ Solicitud recibida. Generando video para: '{prompt}'")

        # Simulación de la generación del video
        video_filename = "video_generado.mp4"
        video_path = f"/content/{video_filename}"
        with open(video_path, "w") as f:
            f.write(f"Este es un video de prueba para el prompt: {prompt}")
        print(f"✔️ Video simulado creado en: {video_path}")

        # Subir el video a Cloudinary
        print("☁️ Subiendo video a Cloudinary...")
        upload_result = cloudinary.uploader.upload_video(video_path, resource_type="video")
        public_url = upload_result['secure_url']
        print(f"✔️ Subida completada. URL: {public_url}")

        os.remove(video_path) # Limpiar el archivo local

        return {"status": "éxito", "video_url": public_url}

    except Exception as e:
        print(f"❌ Ocurrió un error grave en el endpoint: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# ==============================================================================
#  PASO 3: Iniciar el túnel ngrok y arrancar el servidor
# ==============================================================================
nest_asyncio.apply()

# Configurar el token de autenticación de ngrok desde los Secrets
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ Token de autenticación de ngrok configurado.")
except Exception as e:
    print("❌ No se pudo encontrar el NGROK_AUTH_TOKEN en los Secrets de Colab.")

# Iniciar el túnel
COLAB_PORT = 8000
ngrok.kill() # Cierra túneles anteriores
public_url = ngrok.connect(COLAB_PORT).public_url
print("======================================================================")
print(f"🔥 TU SERVIDOR ESTÁ LISTO Y EN LÍNEA EN ESTA URL PÚBLICA:")
print(f"   {public_url}")
print("======================================================================")

# Arrancar el servidor FastAPI
uvicorn.run(app, host="0.0.0.0", port=COLAB_PORT)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 37.0 MB/s eta 0:00:00


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

✅ Configuración de Cloudinary exitosa.
✅ Token de autenticación de ngrok configurado.
🔥 TU SERVIDOR ESTÁ LISTO Y EN LÍNEA EN ESTA URL PÚBLICA:
   https://4cdb2a237b32.ngrok-free.app


INFO:     Started server process [622]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
